# Import Library

In [1]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd

# Load Env Variables

In [2]:
load_dotenv()

API_VERSION = "v19.0"
ACCESS_TOKEN = os.environ.get("ACCESS_TOKEN", "")
AD_ACCOUNT_ID = os.environ.get("AD_ACCOUNT_ID", "")

# Get metrics of the API by days

In [3]:
url = f"https://graph.facebook.com/{API_VERSION}/act_{AD_ACCOUNT_ID}/insights"

params = {
    "fields": "ad_name, adset_name, campaign_name, spend, impressions, reach, cost_per_conversion, frequency, inline_link_clicks, ctr, cpm, cpc, clicks, date_start, ad_id",
    "date_preset": "maximum",
    "time_increment": 1,
    "limit": 100,  # Adjust as needed
    "level": "ad"
}
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

# Get the data and display it into a table by days

In [4]:
insights = []

while True:
    r = requests.get(url=url, headers=headers, params=params)
    insight_data = r.json()

    if 'error' in insight_data:
        print("Error:", insight_data['error'])
        break

    insights.extend(insight_data['data'])

    # Check for pagination
    if 'paging' in insight_data and 'cursors' in insight_data['paging'] and 'after' in insight_data['paging']['cursors']:
        params['after'] = insight_data['paging']['cursors']['after']
    else:
        break

df_day = pd.DataFrame(insights)
df_day.rename(columns=
                    {"ad_name": "Ad Name", "adset_name": "Adset Name", 
                      "campaign_name": "Campaign Name", "spend": "Amount Spend", 
                      "impressions": "Impressions", "reach": "Reach",
                      "frequency": "Frequency", "ctr": "CTR(all)",
                      "clicks": "Clicks(all)", "cpm": "CPM(cost per 1000 impressions)",
                      "cpc": "CPC(all)", "inline_link_clicks": "Link Clicks",
                      "date_start": "Day", "ad_id": "Ad ID"
                    },
                 inplace=True
                )
df_day.drop(columns="date_stop", inplace=True)
df_day.head()
df_day

,Ad Name,Adset Name,Campaign Name,Amount Spend,Impressions,Reach,Frequency,CTR(all),CPM(cost per 1000 impressions),CPC(all),Clicks(all),Day,Ad ID,Link Clicks
0,New Traffic ad,New Traffic ad set,Traffic to Discord #1,0.34,981,980,1.00102,0.101937,0.346585,0.34,1,2023-01-18,23853071466000019,NaN
1,New Traffic ad,New Traffic ad set – Copy,Traffic to Discord #1,0.18,415,414,1.002415,0.240964,0.433735,0.18,1,2023-01-18,23853072122590019,1
2,New Traffic ad,New Traffic ad set,Traffic to Discord #1,1.66,4660,4655,1.001074,0.042918,0.356223,0.83,2,2023-01-19,23853071466000019,1
3,New Traffic ad,New Traffic ad set – Copy,Traffic to Discord #1,1.57,3864,3559,1.085698,0.439959,0.406315,0.092353,17,2023-01-19,23853072122590019,11
4,New Traffic ad,New Traffic ad set,Traffic to Discord #1,0.98,2962,2957,1.001691,0.101283,0.330858,0.326667,3,2023-01-20,23853071466000019,1


# Get metrics of the API by months

In [5]:
url = f"https://graph.facebook.com/{API_VERSION}/act_{AD_ACCOUNT_ID}/insights"

params = {
    "fields": "ad_name, adset_name, campaign_name, spend, impressions, reach, cost_per_conversion, frequency, inline_link_clicks, ctr, cpm, cpc, clicks, date_start, date_stop, ad_id",
    "time_increment": "monthly",
    "date_preset": "maximum",
    "limit": 100,  # Adjust as needed
    "level": "ad"
}
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

# Get the metrics and display it into a table by months

In [6]:
insights = []

while True:
    r = requests.get(url=url, headers=headers, params=params)
    insight_data = r.json()

    if 'error' in insight_data:
        print("Error:", insight_data['error'])
        break

    insights.extend(insight_data['data'])

    # Check for pagination
    if 'paging' in insight_data and 'cursors' in insight_data['paging'] and 'after' in insight_data['paging']['cursors']:
        params['after'] = insight_data['paging']['cursors']['after']
    else:
        break

df_month = pd.DataFrame(insights)
df_month.rename(columns=
                    {"ad_name": "Ad Name", "adset_name": "Adset Name", 
                      "campaign_name": "Campaign Name", "spend": "Amount Spend", 
                      "impressions": "Impressions", "reach": "Reach",
                      "frequency": "Frequency", "ctr": "CTR(all)",
                      "clicks": "Clicks(all)", "cpm": "CPM(cost per 1000 impressions)",
                      "cpc": "CPC(All)", "inline_link_clicks": "Link Clicks",
                      "ad_id": "Ad ID"
                    },
                 inplace=True
                )
df_month["month"] = df_month["date_start"] + " - " + df_month["date_stop"]
df_month.drop(columns=["date_stop", "date_start"], inplace=True)

df_month.head()
df_month

,Ad Name,Adset Name,Campaign Name,Amount Spend,Impressions,Reach,Frequency,Link Clicks,CTR(all),CPM(cost per 1000 impressions),CPC(All),Clicks(all),Ad ID,month
0,New Traffic ad,New Traffic ad set,Traffic to Discord #1,2.98,8603,7779,1.105926,2,0.069743,0.346391,0.496667,6,23853071466000019,2023-01-01 - 2023-01-31
1,New Traffic ad,New Traffic ad set – Copy,Traffic to Discord #1,2.97,7492,6270,1.194896,18,0.373732,0.396423,0.106071,28,23853072122590019,2023-01-01 - 2023-01-31
2,Awareness - Europe - Image,Awareness ad set – Farfetch & Highsnobiety - USA,Awareness campaign,0,24,24,1,NaN,0,0,NaN,0,23853877010110019,2023-03-01 - 2023-03-31
3,Awareness - Europe - Carousel,Awareness ad set – Farfetch & Highsnobiety - USA,Awareness campaign,0,13,13,1,NaN,0,0,NaN,0,23853877018180019,2023-03-01 - 2023-03-31
4,Awareness - Europe - Image,Awareness ad set – Farfetch & Highsnobiety - Asia,Awareness campaign,66.15,762473,731267,1.042674,1006,0.212467,0.086757,0.040833,1620,23853877506500019,2023-03-01 - 2023-03-31


# Get country of the API by months

In [7]:
url = f"https://graph.facebook.com/{API_VERSION}/act_{AD_ACCOUNT_ID}/insights"

params = {
    "fields": "ad_name, adset_name, campaign_name, date_start, date_stop, ad_id",
    "time_increment": "monthly",
    "limit": 100,  # Adjust as needed
    "level": "ad",
    "date_preset": "maximum",
    "breakdowns": ["country"]
}
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

# Get the country and display it into a table by months

In [8]:
insights = []

while True:
    r = requests.get(url=url, headers=headers, params=params)
    insight_data = r.json()

    if 'error' in insight_data:
        print("Error:", insight_data['error'])
        break

    insights.extend(insight_data['data'])

    # Check for pagination
    if 'paging' in insight_data and 'cursors' in insight_data['paging'] and 'after' in insight_data['paging']['cursors']:
        params['after'] = insight_data['paging']['cursors']['after']
    else:
        break

df_country_month = pd.DataFrame(insights)
df_country_month.rename(columns=
                    {"ad_name": "Ad Name", "adset_name": "Adset Name", 
                      "campaign_name": "Campaign Name", "country": "Country",
                      "ad_id": "Ad ID"
                    },
                 inplace=True
                )
df_country_month["Month"] = df_country_month["date_start"] + " - " + df_country_month["date_stop"]
df_country_month.drop(columns=["date_start", "date_stop"])
df_country_month

,Ad Name,Adset Name,Campaign Name,date_start,date_stop,Ad ID,Country,Month
0,New Traffic ad,New Traffic ad set,Traffic to Discord #1,2023-01-01,2023-01-31,23853071466000019,HK,2023-01-01 - 2023-01-31
1,New Traffic ad,New Traffic ad set,Traffic to Discord #1,2023-01-01,2023-01-31,23853071466000019,IT,2023-01-01 - 2023-01-31
2,New Traffic ad,New Traffic ad set,Traffic to Discord #1,2023-01-01,2023-01-31,23853071466000019,JP,2023-01-01 - 2023-01-31
3,New Traffic ad,New Traffic ad set,Traffic to Discord #1,2023-01-01,2023-01-31,23853071466000019,LU,2023-01-01 - 2023-01-31
4,New Traffic ad,New Traffic ad set,Traffic to Discord #1,2023-01-01,2023-01-31,23853071466000019,NL,2023-01-01 - 2023-01-31
...,...,...,...,...,...,...,...,...
1168,Awareness - Europe - Image,Awareness ad set – Farfetch & Highsnobiety Europe,Awareness campaign,2023-05-01,2023-05-31,23853877506590019,SJ,2023-05-01 - 2023-05-31
1169,Awareness - Europe - Image,Awareness ad set – Farfetch & Highsnobiety Europe,Awareness campaign,2023-05-01,2023-05-31,23853877506590019,SE,2023-05-01 - 2023-05-31
1170,Awareness - Europe - Image,Awareness ad set – Farfetch & Highsnobiety Europe,Awareness campaign,2023-05-01,2023-05-31,23853877506590019,CH,2023-05-01 - 2023-05-31
1171,Awareness - Europe - Image,Awareness ad set – Farfetch & Highsnobiety Europe,Awareness campaign,2023-05-01,2023-05-31,23853877506590019,UA,2023-05-01 - 2023-05-31


# Get regions of the API by months

In [9]:
url = f"https://graph.facebook.com/{API_VERSION}/act_{AD_ACCOUNT_ID}/insights"

params = {
    "fields": "ad_name, adset_name, campaign_name, date_start, date_stop, ad_id",
    "time_increment": "monthly",
    "limit": 100,  # Adjust as needed
    "level": "ad",
    "date_preset": "maximum",
    "breakdowns": ["region"]
}
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

# Get the regions and display it into a table by months

In [10]:
insights = []

while True:
    r = requests.get(url=url, headers=headers, params=params)
    insight_data = r.json()

    if 'error' in insight_data:
        print("Error:", insight_data['error'])
        break

    insights.extend(insight_data['data'])

    # Check for pagination
    if 'paging' in insight_data and 'cursors' in insight_data['paging'] and 'after' in insight_data['paging']['cursors']:
        params['after'] = insight_data['paging']['cursors']['after']
    else:
        break

df_region_month = pd.DataFrame(insights)
df_region_month.rename(columns=
                    {"ad_name": "Ad Name", "adset_name": "Adset Name", 
                      "campaign_name": "Campaign Name", "region": "Region",
                      "ad_id": "Ad ID"
                    },
                 inplace=True
                )
df_region_month["Month"] = df_region_month["date_start"] + " - " + df_region_month["date_stop"]
df_region_month.drop(columns=["date_start", "date_stop"])
df_region_month

ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))